In [19]:
import requests
import json

# URLからJSONデータを取得
url = "http://www.jma.go.jp/bosai/common/const/area.json"
request = requests.get(url)

# JSONデータをPythonの辞書として解析
area_json = request.json()

# "centers" キー内のデータを取得
centers = area_json.get("centers", {})

# すべてのエリア情報を表示
for area_code, area_info in centers.items():
    name = area_info.get("name", "N/A")
    enName = area_info.get("enName", "N/A")
    officeName = area_info.get("officeName", "N/A")
    children = area_info.get("children", [])

    print(f"エリアコード: {area_code}")
    print(f"エリア名: {name}")
    print(f"英名: {enName}")
    print(f"事務所名: {officeName}")
    print(f"子エリアID: {', '.join(children) if children else 'N/A'}")
    print("-" * 50)  # 区切り線を表示


エリアコード: 010100
エリア名: 北海道地方
英名: Hokkaido
事務所名: 札幌管区気象台
子エリアID: 011000, 012000, 013000, 014030, 014100, 015000, 016000, 017000
--------------------------------------------------
エリアコード: 010200
エリア名: 東北地方
英名: Tohoku
事務所名: 仙台管区気象台
子エリアID: 020000, 030000, 040000, 050000, 060000, 070000
--------------------------------------------------
エリアコード: 010300
エリア名: 関東甲信地方
英名: Kanto Koshin
事務所名: 気象庁
子エリアID: 080000, 090000, 100000, 110000, 120000, 130000, 140000, 190000, 200000
--------------------------------------------------
エリアコード: 010400
エリア名: 東海地方
英名: Tokai
事務所名: 名古屋地方気象台
子エリアID: 210000, 220000, 230000, 240000
--------------------------------------------------
エリアコード: 010500
エリア名: 北陸地方
英名: Hokuriku
事務所名: 新潟地方気象台
子エリアID: 150000, 160000, 170000, 180000
--------------------------------------------------
エリアコード: 010600
エリア名: 近畿地方
英名: Kinki
事務所名: 大阪管区気象台
子エリアID: 250000, 260000, 270000, 280000, 290000, 300000
--------------------------------------------------
エリアコード: 010700
エリア名: 中国地方（山口県を除く）
英名: Chug

In [25]:
import requests
import json
import csv

# URLからJSONデータを取得
url = "http://www.jma.go.jp/bosai/common/const/area.json"
request = requests.get(url)

# JSONデータをPythonの辞書として解析
area_json = request.json()

# "centers" キー内のデータを取得
centers = area_json.get("centers", {})

# CSVファイルを開く（書き込みモード）
with open('area_info.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # CSVのヘッダー行を作成（idを追加）
    writer.writerow(['id', 'エリアコード', 'エリア名', '英名', '事務所名', '子エリアID'])

    # idの初期化
    id_counter = 1

    # 各エリア情報をCSV形式で書き込む
    for area_code, area_info in centers.items():
        name = area_info.get("name", "N/A")
        enName = area_info.get("enName", "N/A")
        officeName = area_info.get("officeName", "N/A")
        children = area_info.get("children", [])

        # 親エリアの情報をCSVに書き込む（子エリアIDは後で処理）
        if children:
            for region_code in children:
                # 各行にidを追加
                writer.writerow([id_counter, area_code, name, enName, officeName, region_code])
                id_counter += 1
        else:
            # 子エリアがいない場合は1行にまとめて書き込む
            writer.writerow([id_counter, area_code, name, enName, officeName, 'N/A'])
            id_counter += 1

print("CSVファイルに書き込みが完了しました。")


CSVファイルに書き込みが完了しました。


In [27]:
import requests
import json

# URLからJSONデータを取得
url = "http://www.jma.go.jp/bosai/common/const/area.json"
request = requests.get(url)

# JSONデータをPythonの辞書として解析
area_json = request.json()

# "centers" キー内のデータを取得
centers = area_json.get("centers", {})

# 各エリアの子エリアIDを表示（親エリア名やIDは表示しない）
for area_code, area_info in centers.items():
    children = area_info.get("children", [])
    
    if children:
        for region_code in children:
            print(region_code)  # 子エリアIDのみ表示


011000
012000
013000
014030
014100
015000
016000
017000
020000
030000
040000
050000
060000
070000
080000
090000
100000
110000
120000
130000
140000
190000
200000
210000
220000
230000
240000
150000
160000
170000
180000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
360000
370000
380000
390000
350000
400000
410000
420000
430000
440000
450000
460040
460100
471000
472000
473000
474000
